In [15]:
!pip install pyLDAvis
!pip install contractions autocorrect vaderSentiment bertopic plotly joblib emoji gensim umap-learn pyLDAvis transformers torch

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
import pandas as pd
import re
import nltk
import emoji
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from bertopic import BERTopic
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from transformers import BertTokenizer, BertModel
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, GRU, Bidirectional, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, SimpleRNN
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score
import torch
import gensim
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
from wordcloud import WordCloud
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [17]:
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification

# Load the tokenizer and the fine-tuned model
model_path = '/content/drive/MyDrive/finetuned_model'
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
model = TFDistilBertForSequenceClassification.from_pretrained(model_path)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Some layers from the model checkpoint at /content/drive/MyDrive/finetuned_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_79']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForS

In [18]:
class SentimentChatbot:
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model

    def analyze_sentiment(self, text):
        # Tokenize input text
        inputs = self.tokenizer(text, return_tensors='tf', truncation=True, padding=True)
        # Get model predictions
        outputs = self.model(inputs)
        # Get the predicted class (logits)
        logits = outputs.logits
        sentiment = tf.argmax(logits, axis=1).numpy()[0]
        return sentiment

    def respond(self, user_input):
        # Analyze the sentiment of the user input
        sentiment = self.analyze_sentiment(user_input)
        # Map the sentiment class to a human-readable response
        sentiment_map = {0: "Negative", 1: "Positive", 2: "Neutral"}
        sentiment_text = sentiment_map[sentiment]
        return f"The sentiment of the given tweet is: {sentiment_text}"

# Initialize the chatbot with the tokenizer and model
chatbot = SentimentChatbot(tokenizer, model)

# Example usage
user_input = "I love using this new app, it's fantastic!"
response = chatbot.respond(user_input)
print(response)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


The sentiment of the given tweet is: Positive


In [19]:
# Example usage
user_input = "Flood is devasting"
response = chatbot.respond(user_input)
print(response)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


The sentiment of the given tweet is: Neutral


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
